In [ ]:
# %% Deep learning - Section 19.174
#    CNN to classify MNIST digits

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [2]:
# %% Libraries and modules
import numpy                  as np
import matplotlib.pyplot      as plt
import torch
import torch.nn               as nn
import seaborn                as sns
import copy
import torch.nn.functional    as F
import pandas                 as pd
import scipy.stats            as stats
import sklearn.metrics        as skm
import time
import sys
import imageio.v2             as imageio
import torchvision
import torchvision.transforms as T

from torch.utils.data                 import DataLoader,TensorDataset,Dataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from scipy.stats                      import zscore
from sklearn.decomposition            import PCA
from scipy.signal                     import convolve2d
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.style.use('default')


In [5]:
# %% Data

# Load data
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Split labels from data
labels = data[:,0]
data   = data[:,1:]

# Normalise data (original range is (0,255))
data_norm = data / np.max(data)

# New here: reshape to 2D actual images
data_norm = data_norm.reshape(data_norm.shape[0],1,28,28)


In [ ]:
# %% Check size

print(data_norm.shape)


In [8]:
# %% Create train and test datasets

# Convert to tensor (float and integers)
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Check size

# Should be (images x channels x width x height)
print(train_loader.dataset.tensors[0].shape)
print(test_loader.dataset.tensors[0].shape)


In [14]:
# %% Function to generate the model

def gen_model(printing_toggle=False):

    class mnist_CNN(nn.Module):
        def __init__(self,printing_toggle):
            super().__init__()

            # Convolution layer 1
            # size = np.floor( (28+2*1-5)/1 )+1 = 26/2 = 13 (divide by 2 because
            # will have maxpool with extent 2)
            self.conv1 = nn.Conv2d(1,10,kernel_size=5,stride=1,padding=1)

            # Convolution layer 2
            # size = np.floor( (13+2*1-5)/1 )+1 = 11/2 = 5 (divide by 2 because
            # will have maxpool with extent 2)
            self.conv2 = nn.Conv2d(10,20,kernel_size=5,stride=1,padding=1)

            # Number of units expected in fully connected layer (out of conv2);
            # note that c1 layer has no padding nor kernel, so we set those
            # params to be 0 and 1 respectively; we can also square because the
            # images are squares
            expected_size = np.floor( 5+2*0-1 ) + 1
            expected_size = 20*int(expected_size**2)

            # Fully connected layer
            self.fc1 = nn.Linear(expected_size,50)

            # Output layer
            self.output = nn.Linear(50,10)

            # Toggle for the printing of tensor sizes during forward propagation
            self.print = printing_toggle

        def forward(self,x):

            # Print input layer size
            print(f'Input size: {x.shape}') if self.print else None

            # MaxPool and ReLu on convolution layer 1
            x = F.relu(F.max_pool2d(self.conv1(x),2))
            print(f'Conv. layer 1 size: {x.shape}') if self.print else None

            # MaxPool and ReLu on convolution layer 2
            x = F.relu(F.max_pool2d(self.conv2(x),2))
            print(f'Conv. layer 2 size: {x.shape}') if self.print else None

            # Vectorise for linear layer
            n_units = x.shape.numel() / x.shape[0]
            x       = x.view(-1,int(n_units))
            print(f'Vectorised conv. 2 layer size: {x.shape}') if self.print else None

            # Linear and output layers
            x = F.relu(self.fc1(x))
            print(f'Linear layer size: {x.shape}') if self.print else None
            x = self.output(x)
            print(f'Output layer size: {x.shape}') if self.print else None

            return x

    # Create model instance
    CNN = mnist_CNN(printing_toggle)

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.Adam(CNN.parameters(),lr=0.001)

    return CNN,loss_fun,optimizer


In [ ]:
# %% Test the model on one batch

CNN,loss_fun,optimizer = gen_model(printing_toggle=True)

X,y  = next(iter(train_loader))
yHat = CNN(X)

# Check sizes of output and target variable
print()
print(yHat.shape), print()
print(y.shape), print()

# Check loss
loss = loss_fun(yHat,y)
print(loss)


In [ ]:
# %% Check all the parameters in the model

summary(CNN,(1,28,28))


In [39]:
# %% Function to train the model

# Nothing new on the western front

def train_model():

    # Parameters, model instance, inizialise vars
    num_epochs = 10
    CNN,loss_fun,optimizer = gen_model()

    losses    = []
    train_acc = []
    test_acc  = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = CNN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        # Test accuracy
        CNN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = CNN(X)
            test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        CNN.train()

    return train_acc,test_acc,losses,CNN


In [37]:
# %% Run the model

train_acc,test_acc,losses,CNN = train_model()


In [ ]:
# Plotting

# Remember how the FFN could never really go higher than ~95% ?
phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*phi*5,5))

ax[0].plot(losses,'s-')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_title('Model loss')

ax[1].plot(train_acc,'s-',label='Train')
ax[1].plot(test_acc,'o-',label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_title(f'Final model test accuracy: {test_acc[-1]:.2f}%')
ax[1].legend()

plt.savefig('figure1_mnist_cnn.png')
plt.show()
files.download('figure1_mnist_cnn.png')


In [31]:
# %% Exercise 1
#    Do we need both convolution layers in this model? Comment out the "conv2" layers in the mode definition. What else
#    needs to be changed in the code for this to work with one convolution layer? Once you get it working, how does the
#    accuracy compare between one and two conv layers? (hint: perhaps try adding some more training epochs)

# Function to generate the model
def gen_model(printing_toggle=False):

    class mnist_CNN(nn.Module):
        def __init__(self,printing_toggle):
            super().__init__()

            # Convolution layer 1
            self.conv1 = nn.Conv2d(1,10,kernel_size=5,stride=1,padding=1)

            # Number of units expected in fully connected layer (out of conv1)
            expected_size = np.floor( 13+2*0-1 ) + 1
            expected_size = 10*int(expected_size**2)

            # Fully connected layer
            self.fc1 = nn.Linear(expected_size,50)

            # Output layer
            self.output = nn.Linear(50,10)

            # Toggle for the printing of tensor sizes during forward propagation
            self.print = printing_toggle

        def forward(self,x):

            # Print input layer size
            print(f'Input size: {x.shape}') if self.print else None

            # MaxPool and ReLu on convolution layer 1
            x = F.relu(F.max_pool2d(self.conv1(x),2))
            print(f'Conv. layer 1 size: {x.shape}') if self.print else None

            # Vectorise for linear layer
            n_units = x.shape.numel() / x.shape[0]
            x       = x.view(-1,int(n_units))
            print(f'Vectorised conv. 1 layer size: {x.shape}') if self.print else None

            # Linear and output layers
            x = F.relu(self.fc1(x))
            print(f'Linear layer size: {x.shape}') if self.print else None
            x = self.output(x)
            print(f'Output layer size: {x.shape}') if self.print else None

            return x

    # Create model instance
    CNN = mnist_CNN(printing_toggle)

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.Adam(CNN.parameters(),lr=0.001)

    return CNN,loss_fun,optimizer
